In [10]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary

In [18]:
import tensorflow as tf
from tensorflow.keras.datasets.cifar10 import load_data

In [12]:
# 다음 배치를 읽어오기 위한 next_batch 유틸리티 함수를 정의합니다.
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [13]:
# CNN 모델을 정의합니다. 
def build_CNN_classifier(x):
    
  """CIFAR-10 이미지를 분류하기 위한 Convolutional Neural Networks 그래프를 생성합니다.
  인자들(Args):
    x: (N_examples, 32, 32, 3) 차원을 가진 input tensor, CIFAR-10 데이터는 32x32 크기의 컬러이미지이다.
  리턴값들(Returns):
    tuple (y, keep_prob). y는 (N_examples, 10)형태의 숫자(0-9) tensor이다. 
    keep_prob는 dropout을 위한 scalar placeholder이다.
  """

  # 입력 이미지
  x_image = x

  # 첫번째 convolutional layer - 하나의 grayscale 이미지를 64개의 특징들(feature)으로 맵핑(maping)한다.
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], stddev=5e-2))
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

  # 첫번째 Pooling layer
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  # 두번째 convolutional layer -- 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)한다.
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

  # 두번째 pooling layer.
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  # 세번째 convolutional layer
  W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
  b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

  # 네번째 convolutional layer
  W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
  h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

  # 다섯번째 convolutional layer
  W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)

  # Fully Connected Layer 1 -- 2번의 downsampling 이후에, 우리의 32x32 이미지는 8x8x128 특징맵(feature map)이 된다.
  # 이를 384개의 특징들로 맵핑(maping)한다.
  W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

  h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
  h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

  # Dropout - 모델의 복잡도를 컨트롤한다. 특징들의 co-adaptation을 방지한다.
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

  # 384개의 특징들(feature)을 10개의 클래스-airplane, automobile, bird...-로 맵핑(maping)한다.
  W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2))
  b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
  logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
  y_pred = tf.nn.softmax(logits)

  return y_pred, logits

In [19]:
# 인풋 아웃풋 데이터, 드롭아웃 확률을 입력받기위한 플레이스홀더를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

# CIFAR-10 데이터를 다운로드하고 데이터를 불러옵니다.
(x_train, y_train), (x_test, y_test) = load_data()

# scalar 형태의 레이블(0~9)을 One-hot Encoding 형태로 변환합니다.
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

# Convolutional Neural Networks(CNN) 그래프를 생성합니다.
y_pred, logits = build_CNN_classifier(x)

# Cross Entropy를 비용함수(loss function)으로 정의하고, RMSPropOptimizer를 이용해서 비용 함수를 최소화합니다.
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

# 정확도를 계산하는 연산을 추가합니다.
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

170500096/170498071 [==============================] - 172s 1us/step
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [29]:
# 세션을 열어 실제 학습을 진행합니다.
with tf.Session() as sess:
  # 모든 변수들을 초기화한다. 
  sess.run(tf.global_variables_initializer())
  
  # 10000 Step만큼 최적화를 수행합니다.
  for i in range(10000):
    batch = next_batch(128, x_train, y_train_one_hot.eval())

    # 100 Step마다 training 데이터셋에 대한 정확도와 loss를 출력합니다.
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
      loss_print = loss.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})

      print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f, 손실 함수(loss): %f" % (i, train_accuracy, loss_print))
      print("Current time is : ", datetime.datetime.now())
    # 20% 확률의 Dropout을 이용해서 학습을 진행한다.
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1], keep_prob: 0.8})

  # 학습이 끝나면 테스트 데이터에 대한 정확도를 출력합니다.  
  test_batch = next_batch(10000, x_test, y_test_one_hot.eval())
  print("테스트 데이터 정확도: %f" % accuracy.eval(feed_dict={x: test_batch[0], y: test_batch[1], keep_prob: 1.0}))
  

반복(Epoch): 0, 트레이닝 데이터 정확도: 0.109375, 손실 함수(loss): 127.778107
Current time is :  2018-08-03 23:51:26.331499
반복(Epoch): 100, 트레이닝 데이터 정확도: 0.101562, 손실 함수(loss): 2.406459
Current time is :  2018-08-03 23:52:23.712072
반복(Epoch): 200, 트레이닝 데이터 정확도: 0.289062, 손실 함수(loss): 1.992114
Current time is :  2018-08-03 23:53:20.867847
반복(Epoch): 300, 트레이닝 데이터 정확도: 0.171875, 손실 함수(loss): 2.135999
Current time is :  2018-08-03 23:54:17.855224
반복(Epoch): 400, 트레이닝 데이터 정확도: 0.437500, 손실 함수(loss): 1.538427
Current time is :  2018-08-03 23:55:14.958698
반복(Epoch): 500, 트레이닝 데이터 정확도: 0.367188, 손실 함수(loss): 1.697785
Current time is :  2018-08-03 23:56:11.985455
반복(Epoch): 600, 트레이닝 데이터 정확도: 0.351562, 손실 함수(loss): 1.598925
Current time is :  2018-08-03 23:57:08.864140
반복(Epoch): 700, 트레이닝 데이터 정확도: 0.437500, 손실 함수(loss): 1.455483
Current time is :  2018-08-03 23:58:05.808136
반복(Epoch): 800, 트레이닝 데이터 정확도: 0.460938, 손실 함수(loss): 1.409690
Current time is :  2018-08-03 23:59:02.889282
반복(Epoch): 900, 트레이닝 데이터 정확도

반복(Epoch): 8000, 트레이닝 데이터 정확도: 0.703125, 손실 함수(loss): 0.790747
Current time is :  2018-08-04 01:07:37.336332
반복(Epoch): 8100, 트레이닝 데이터 정확도: 0.679688, 손실 함수(loss): 0.865468
Current time is :  2018-08-04 01:08:34.306680
반복(Epoch): 8200, 트레이닝 데이터 정확도: 0.726562, 손실 함수(loss): 0.864203
Current time is :  2018-08-04 01:09:31.427252
반복(Epoch): 8300, 트레이닝 데이터 정확도: 0.671875, 손실 함수(loss): 0.977429
Current time is :  2018-08-04 01:10:28.531868
반복(Epoch): 8400, 트레이닝 데이터 정확도: 0.640625, 손실 함수(loss): 1.256478
Current time is :  2018-08-04 01:11:25.588224
반복(Epoch): 8500, 트레이닝 데이터 정확도: 0.765625, 손실 함수(loss): 0.980586
Current time is :  2018-08-04 01:12:22.668208
반복(Epoch): 8600, 트레이닝 데이터 정확도: 0.664062, 손실 함수(loss): 1.004501
Current time is :  2018-08-04 01:13:19.840883
반복(Epoch): 8700, 트레이닝 데이터 정확도: 0.656250, 손실 함수(loss): 0.964734
Current time is :  2018-08-04 01:14:16.808556
반복(Epoch): 8800, 트레이닝 데이터 정확도: 0.757812, 손실 함수(loss): 0.782508
Current time is :  2018-08-04 01:15:13.973059
반복(Epoch): 8900, 트레

In [26]:
import datetime

In [27]:
print(datetime.datetime.now())

2018-08-03 23:49:43.086908
